In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [94]:
bus = pd.read_csv("/Users/kimjisu/Desktop/seoulbigcam/Raw Data/서울시 대중교통시설 위치정보_bus.csv", encoding="CP949")
subway = pd.read_csv("/Users/kimjisu/Desktop/seoulbigcam/Raw Data/서울시 대중교통시설 위치정보_sub.csv", encoding="CP949")

In [95]:
subway.head()

,`GU_NM`,`GU_CD`,`SUB_STA_SN`,`KOR_SUB_NM`,`POINT_X`,`POINT_Y`
0,`중랑구`,`11260`,`40`,`용마산역`,`207607`,`452624`
1,`강북구`,`11305`,`41`,`미아삼거리역`,`202663`,`457082`
2,`강북구`,`11305`,`42`,`수유역`,`202238`,`459793`
3,`강북구`,`11305`,`43`,`미아역`,`202304`,`458542`
4,`강동구`,`11740`,`44`,`상일동역`,`214745`,`450818`


##### csv 파일 전처리

In [96]:
column_name = ['`GU_NM`', '`GU_CD`', '`SUB_STA_SN`', '`KOR_SUB_NM`', '`POINT_X`', '`POINT_Y`']

for i in column_name:
    subway[i] = subway[i].str[1:-1]

subway.head()

,`GU_NM`,`GU_CD`,`SUB_STA_SN`,`KOR_SUB_NM`,`POINT_X`,`POINT_Y`
0,중랑구,11260,40,용마산역,207607,452624
1,강북구,11305,41,미아삼거리역,202663,457082
2,강북구,11305,42,수유역,202238,459793
3,강북구,11305,43,미아역,202304,458542
4,강동구,11740,44,상일동역,214745,450818


In [97]:
subway = subway[['`GU_NM`', '`KOR_SUB_NM`', '`POINT_X`', '`POINT_Y`']]

In [98]:
subway = subway.rename(columns={'`GU_NM`': 'gu',
                               '`KOR_SUB_NM`': 'sub_name',
                               '`POINT_X`':'X_VALUE',
                               '`POINT_Y`':'Y_VALUE'})
subway.head()


,gu,sub_name,X_VALUE,Y_VALUE
0,중랑구,용마산역,207607,452624
1,강북구,미아삼거리역,202663,457082
2,강북구,수유역,202238,459793
3,강북구,미아역,202304,458542
4,강동구,상일동역,214745,450818


In [99]:
bus.columns

Index(['`YYYY_MM`', '`LINE_NO_EXP`', '`LINE_NO`', '`LINE_EXP`', '`SEQ_NO`',
       '`BUS_STA_NM`', '`X_COORD`', '`Y_COORD`', '`ARSID`'],
      dtype='object')

In [100]:
column_name2 = ['`YYYY_MM`', '`LINE_NO_EXP`', '`LINE_NO`', '`LINE_EXP`', '`SEQ_NO`',
       '`BUS_STA_NM`', '`X_COORD`', '`Y_COORD`', '`ARSID`']

for i in column_name2:
    bus[i] = bus[i].str[1:-1]

bus.head()

,`YYYY_MM`,`LINE_NO_EXP`,`LINE_NO`,`LINE_EXP`,`SEQ_NO`,`BUS_STA_NM`,`X_COORD`,`Y_COORD`,`ARSID`
0,201810,105번(상계동~서울역),105번,상계동~서울역,85,상계주공7단지.광림교회앞,127.06278,37.6605,11188
1,201810,1129번(상계현대2차아파트~창동역동측),1129번,상계현대2차아파트~창동역동측,6,상계주공7단지.광림교회앞,127.06278,37.6605,11188
2,201810,146번(상계주공7단지~강남역),146번,상계주공7단지~강남역,133,상계주공7단지.광림교회앞,127.06278,37.6605,11188
3,201810,N13번(상계동차고지~송파공영차고지),N13번,상계동차고지~송파공영차고지,147,상계주공7단지.광림교회앞,127.06278,37.6605,11188
4,201810,N13번(송파공영차고지~상계동차고지),N13번,송파공영차고지~상계동차고지,71,상계주공7단지.광림교회앞,127.06278,37.6605,11188


In [101]:
bus = bus[['`BUS_STA_NM`', '`X_COORD`',	'`Y_COORD`']]
bus = bus.rename(columns={'`BUS_STA_NM`': 'name',
                        '`X_COORD`': 'x',
                        '`Y_COORD`': 'y'})
bus.head()

,name,x,y
0,상계주공7단지.광림교회앞,127.06278,37.6605
1,상계주공7단지.광림교회앞,127.06278,37.6605
2,상계주공7단지.광림교회앞,127.06278,37.6605
3,상계주공7단지.광림교회앞,127.06278,37.6605
4,상계주공7단지.광림교회앞,127.06278,37.6605


### 지하철 데이터 위도 경도 반환

In [88]:
# X,Y좌표 경도 위도로 변환하기

sub = gpd.GeoDataFrame(subway, geometry=gpd.points_from_xy(subway.X_VALUE, subway.Y_VALUE))

sub.crs = 'epsg:5181'
sub = sub.to_crs('epsg:4326')

# 변환 좌표 속성 생성 
sub['x'] = sub['geometry'].x
sub['y'] = sub['geometry'].y

# 필요한 컬럼만 추출해서 저장하기
sub = sub[['sub_name', 'x', 'y']]

print(sub)

    sub_name           x          y
0       용마산역  127.086111  37.573129
1     미아삼거리역  127.030161  37.613322
2        수유역  127.025356  37.637749
3        미아역  127.026100  37.626478
4       상일동역  127.166877  37.556770
..       ...         ...        ...
281      뚝섬역  127.047358  37.547184
282      성수역  127.056081  37.544504
283      옥수역  127.018466  37.541101
284     도봉산역  127.046439  37.689648
285      방학역  127.044340  37.667386

[286 rows x 3 columns]


### REST API로 행정동 추출

In [75]:
import pandas as pd
import requests 
import sys
import json
import datetime 

In [89]:
import pandas as pd
import requests

locations = []

def get_location(row):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(row['x'], row['y'])
    headers = {
        "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"
    }
    response = requests.get(url, headers=headers)
    
    try:
        response_json = response.json()
        place = response_json['documents'][0]
        gu = place.get('region_2depth_name')
        dong = place.get('region_3depth_name')
        locations.append({'gu': gu, 'dong': dong})
    except (IndexError, AttributeError, KeyError):
        print('API 호출 오류:', response.text)

# 데이터프레임의 각 행에 대해 get_location 함수 호출
for i, row in sub.iterrows():
    get_location(row)
    
    # 200개씩 처리될 때 메시지 출력
    if i % 20 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

# 결과를 데이터프레임에 추가
sub['gu'] = [location.get('gu') for location in locations]
sub['dong'] = [location.get('dong') for location in locations]


0 번째 경위도 좌표계에 대한 정보 불러오는 중...
20 번째 경위도 좌표계에 대한 정보 불러오는 중...
40 번째 경위도 좌표계에 대한 정보 불러오는 중...
60 번째 경위도 좌표계에 대한 정보 불러오는 중...
80 번째 경위도 좌표계에 대한 정보 불러오는 중...
100 번째 경위도 좌표계에 대한 정보 불러오는 중...
120 번째 경위도 좌표계에 대한 정보 불러오는 중...
140 번째 경위도 좌표계에 대한 정보 불러오는 중...
160 번째 경위도 좌표계에 대한 정보 불러오는 중...
180 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
220 번째 경위도 좌표계에 대한 정보 불러오는 중...
240 번째 경위도 좌표계에 대한 정보 불러오는 중...
260 번째 경위도 좌표계에 대한 정보 불러오는 중...
280 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [90]:
sub

,sub_name,x,y,gu,dong
0,용마산역,127.086111,37.573129,중랑구,면목동
1,미아삼거리역,127.030161,37.613322,강북구,미아동
2,수유역,127.025356,37.637749,강북구,수유동
3,미아역,127.026100,37.626478,강북구,미아동
4,상일동역,127.166877,37.556770,강동구,상일동
...,...,...,...,...,...
281,뚝섬역,127.047358,37.547184,성동구,성수동1가
282,성수역,127.056081,37.544504,성동구,성수동2가
283,옥수역,127.018466,37.541101,성동구,옥수동
284,도봉산역,127.046439,37.689648,도봉구,도봉동


In [104]:
import pandas as pd
import requests

locations = []

def get_location(row):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(row['x'], row['y'])
    headers = {
        "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"
    }
    response = requests.get(url, headers=headers)
    
    try:
        response_json = response.json()
        place = response_json['documents'][0]
        city = place.get('region_1depth_name')
        gu = place.get('region_2depth_name')
        dong = place.get('region_3depth_name')
        locations.append({'city': city, 'gu': gu, 'dong': dong})
    except (IndexError, AttributeError, KeyError):
        print('API 호출 오류:', response.text)

# 데이터프레임의 각 행에 대해 get_location 함수 호출
for i, row in bus.iterrows():
    get_location(row)
    
    # 100개씩 처리될 때 메시지 출력
    if i % 100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

# 결과를 데이터프레임에 추가
bus['city'] = [location.get('city') for location in locations]
bus['gu'] = [location.get('gu') for location in locations]
bus['dong'] = [location.get('dong') for location in locations]


0 번째 경위도 좌표계에 대한 정보 불러오는 중...
100 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
300 번째 경위도 좌표계에 대한 정보 불러오는 중...
400 번째 경위도 좌표계에 대한 정보 불러오는 중...
500 번째 경위도 좌표계에 대한 정보 불러오는 중...
600 번째 경위도 좌표계에 대한 정보 불러오는 중...
700 번째 경위도 좌표계에 대한 정보 불러오는 중...
800 번째 경위도 좌표계에 대한 정보 불러오는 중...
900 번째 경위도 좌표계에 대한 정보 불러오는 중...
1000 번째 경위도 좌표계에 대한 정보 불러오는 중...
1100 번째 경위도 좌표계에 대한 정보 불러오는 중...
1200 번째 경위도 좌표계에 대한 정보 불러오는 중...
1300 번째 경위도 좌표계에 대한 정보 불러오는 중...
1400 번째 경위도 좌표계에 대한 정보 불러오는 중...
1500 번째 경위도 좌표계에 대한 정보 불러오는 중...
1600 번째 경위도 좌표계에 대한 정보 불러오는 중...
1700 번째 경위도 좌표계에 대한 정보 불러오는 중...
1800 번째 경위도 좌표계에 대한 정보 불러오는 중...
1900 번째 경위도 좌표계에 대한 정보 불러오는 중...
2000 번째 경위도 좌표계에 대한 정보 불러오는 중...
2100 번째 경위도 좌표계에 대한 정보 불러오는 중...
2200 번째 경위도 좌표계에 대한 정보 불러오는 중...
2300 번째 경위도 좌표계에 대한 정보 불러오는 중...
2400 번째 경위도 좌표계에 대한 정보 불러오는 중...
2500 번째 경위도 좌표계에 대한 정보 불러오는 중...
2600 번째 경위도 좌표계에 대한 정보 불러오는 중...
2700 번째 경위도 좌표계에 대한 정보 불러오는 중...
2800 번째 경위도 좌표계에 대한 정보 불러오는 중...
2900 번째 경위도 좌표계에 대한 정보 불러오는 중...
3000 번째 경위도 좌표계에 대한 정보

ConnectionError: ('Connection aborted.', OSError(65, 'No route to host'))

In [ ]:
bus

,name,x,y
0,상계주공7단지.광림교회앞,127.06278,37.6605
1,상계주공7단지.광림교회앞,127.06278,37.6605
2,상계주공7단지.광림교회앞,127.06278,37.6605
3,상계주공7단지.광림교회앞,127.06278,37.6605
4,상계주공7단지.광림교회앞,127.06278,37.6605
...,...,...,...
38418,서울중앙외과,127.09436,37.58781
38419,농협앞,127.09181,37.58941
38420,면목역,127.08697,37.58831
38421,KT중랑지점,127.07818,37.59076
